In [229]:
#!g2.1
%reload_ext autoreload
%autoreload 2

In [48]:
#!g2.1
!git clone https://github.com/Just-ne-Just/llama.git

fatal: destination path 'llama' already exists and is not an empty directory.


Exception: Process exited with code 128

In [200]:
#!g2.1
%cd llama

/home/jupyter/work/resources/llama


In [232]:
#!g2.1
!git pull https://github.com/Just-ne-Just/llama.git

Updating 89e009a..972ca1e
Fast-forward
 src/train/train.py | 2 +-
 1 file changed, 1 insertion(+), 1 deletion(-)


From https://github.com/Just-ne-Just/llama
 * branch            HEAD       -> FETCH_HEAD


In [83]:
#!g2.1
%pip install sentencepiece

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [84]:
#!g2.1
%pip install wandb

Defaulting to user installation because normal site-packages is not writeable
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [202]:
#!g2.1
!nvidia-smi

Sun Dec  3 12:37:13 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.147.05   Driver Version: 525.147.05   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  On   | 00000000:8C:00.0 Off |                    0 |
| N/A   31C    P0    51W / 500W |      5MiB / 81920MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [233]:
#!g2.1
from src.dataset.dataset import TinyStoriesDataset
from src.model.llama import LLAMA
from src.model.utils import create_mask
from src.train.train import train

In [204]:
#!g2.1
train_ds = TinyStoriesDataset(data_file="../dataset/TinyStoriesV3-GPT4-train.txt", sp_model_prefix="./src/tokenizer/bpe/bpe")

Loading tokenizer...
Tokenizer loaded
Loading texts...
Texts loaded
Encoding texts...
Texts encoded


In [205]:
#!g2.1
val_ds = TinyStoriesDataset(data_file="../dataset/TinyStoriesV3-GPT4-valid.txt", sp_model_prefix="./src/tokenizer/bpe/bpe")

Loading tokenizer...
Tokenizer loaded
Loading texts...
Texts loaded
Encoding texts...
Texts encoded


In [206]:
#!g2.1
from torch.utils.data import DataLoader

In [224]:
#!g2.1
train_loader = DataLoader(train_ds, batch_size=512, shuffle=True)

In [208]:
#!g2.1
val_loader = DataLoader(val_ds, batch_size=16, shuffle=False)

In [198]:
#!g2.1
import gc
import torch

gc.collect()
torch.cuda.empty_cache()

for p in model.parameters():
    if p.grad is not None:
        del p.grad
        torch.cuda.empty_cache()

In [209]:
#!g2.1
model = LLAMA(num_layers=8, embed_dim=512, num_heads=8, vocab_size=train_ds.vocab_size, ff_dim=2048, dropout=0.1)

In [210]:
#!g2.1
model = model.to('cuda')

In [211]:
#!g2.1
from torch.optim import AdamW

In [212]:
#!g2.1
optimizer = AdamW(model.parameters(), lr=3e-4, weight_decay=1e-5)

In [213]:
#!g2.1
device = 'cuda'

In [214]:
#!g2.1
from torch.optim.lr_scheduler import OneCycleLR

In [215]:
#!g2.1
scheduler = OneCycleLR(optimizer=optimizer, max_lr=3e-4, epochs=80, steps_per_epoch=2000, pct_start=0.2)

In [216]:
#!g2.1
import warnings
warnings.filterwarnings("ignore")

In [217]:
#!g2.1
train(80, model, train_ds.pad_id, optimizer, train_loader, val_loader, device, train_ds, scheduler=scheduler, len_epoch=2000, log_step=100)

wandb: Currently logged in as: liamstartips (wbot). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/jupyter/.netrc
wandb: Tracking run with wandb version 0.16.0
wandb: Run data is saved locally in /home/jupyter/work/resources/llama/wandb/run-20231203_123900-rwybrt03
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run solar-silence-10
wandb: ⭐️ View project at https://wandb.ai/wbot/llama
wandb: 🚀 View run at https://wandb.ai/wbot/llama/runs/rwybrt03


train:   0%|          | 0/2000 [00:00<?, ?it/s]

Epoch: 1, Train loss: 7.834
Epoch: 1, Train loss: 5.973
Epoch: 1, Train loss: 5.039
Epoch: 1, Train loss: 4.635
Epoch: 1, Train loss: 4.379
Epoch: 1, Train loss: 4.188
Epoch: 1, Train loss: 4.032
Epoch: 1, Train loss: 3.909
Epoch: 1, Train loss: 3.808
Epoch: 1, Train loss: 3.715
Epoch: 1, Train loss: 3.639
Epoch: 1, Train loss: 3.568
Epoch: 1, Train loss: 3.499
Epoch: 1, Train loss: 3.435
Epoch: 1, Train loss: 3.377
Epoch: 1, Train loss: 3.328
Epoch: 1, Train loss: 3.274
Epoch: 1, Train loss: 3.228
Epoch: 1, Train loss: 3.185
Epoch: 1, Train loss: 3.144


  0%|          | 0/1727 [00:00<?, ?it/s]

Saving checkpoint...
Epoch: 1, Train loss: 3.117, Val loss: 2.979


...

Epoch: 5, Train loss: 1.990
Epoch: 5, Train loss: 1.984
Epoch: 5, Train loss: 1.974
Epoch: 5, Train loss: 1.963
Epoch: 5, Train loss: 1.959
Epoch: 5, Train loss: 1.950
Epoch: 5, Train loss: 1.939
Epoch: 5, Train loss: 1.928
Epoch: 5, Train loss: 1.919
Epoch: 5, Train loss: 1.912
Epoch: 5, Train loss: 1.903
Epoch: 5, Train loss: 1.897
Epoch: 5, Train loss: 1.890
Epoch: 5, Train loss: 1.880
Epoch: 5, Train loss: 1.871
Epoch: 5, Train loss: 1.862
Epoch: 5, Train loss: 1.857
Epoch: 5, Train loss: 1.849
Epoch: 5, Train loss: 1.842
Epoch: 5, Train loss: 1.833


  0%|          | 0/1727 [00:00<?, ?it/s]

Saving checkpoint...
Epoch: 5, Train loss: 1.857, Val loss: 1.712


train:   0%|          | 0/2000 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
#!g2.1
train(75, model, train_ds.pad_id, optimizer, train_loader, val_loader, device, train_ds, scheduler=scheduler, len_epoch=2000, log_step=100)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


train:   0%|          | 0/2000 [00:00<?, ?it/s]

Epoch: 1, Train loss: 1.809
Epoch: 1, Train loss: 1.801
Epoch: 1, Train loss: 1.792
Epoch: 1, Train loss: 1.786
Epoch: 1, Train loss: 1.785
Epoch: 1, Train loss: 1.773
Epoch: 1, Train loss: 1.768
Epoch: 1, Train loss: 1.763
Epoch: 1, Train loss: 1.755
Epoch: 1, Train loss: 1.755
Epoch: 1, Train loss: 1.746
Epoch: 1, Train loss: 1.739
Epoch: 1, Train loss: 1.730
Epoch: 1, Train loss: 1.728
Epoch: 1, Train loss: 1.718
Epoch: 1, Train loss: 1.713
Epoch: 1, Train loss: 1.711
Epoch: 1, Train loss: 1.706
Epoch: 1, Train loss: 1.699
Epoch: 1, Train loss: 1.694


  0%|          | 0/1727 [00:00<?, ?it/s]

Saving checkpoint...
Epoch: 1, Train loss: 1.724, Val loss: 1.587


...

Epoch: 24, Train loss: 1.193
Epoch: 24, Train loss: 1.198
Epoch: 24, Train loss: 1.194
Epoch: 24, Train loss: 1.197
Epoch: 24, Train loss: 1.194
Epoch: 24, Train loss: 1.194
Epoch: 24, Train loss: 1.192
Epoch: 24, Train loss: 1.195
Epoch: 24, Train loss: 1.194
Epoch: 24, Train loss: 1.196
Epoch: 24, Train loss: 1.194
Epoch: 24, Train loss: 1.191
Epoch: 24, Train loss: 1.192
Epoch: 24, Train loss: 1.193
Epoch: 24, Train loss: 1.197
Epoch: 24, Train loss: 1.197
Epoch: 24, Train loss: 1.192
Epoch: 24, Train loss: 1.208
Epoch: 24, Train loss: 1.189
Epoch: 24, Train loss: 1.183


  0%|          | 0/1727 [00:00<?, ?it/s]

Saving checkpoint...
Epoch: 24, Train loss: 1.190, Val loss: 1.125


train:   0%|          | 0/2000 [00:00<?, ?it/s]

Epoch: 25, Train loss: 1.186
Epoch: 25, Train loss: 1.186
Epoch: 25, Train loss: 1.182
Epoch: 25, Train loss: 1.184
Epoch: 25, Train loss: 1.183
Epoch: 25, Train loss: 1.184
Epoch: 25, Train loss: 1.184
Epoch: 25, Train loss: 1.185
Epoch: 25, Train loss: 1.188
Epoch: 25, Train loss: 1.184
Epoch: 25, Train loss: 1.185
Epoch: 25, Train loss: 1.184
Epoch: 25, Train loss: 1.186
Epoch: 25, Train loss: 1.185
Epoch: 25, Train loss: 1.186


In [ ]:
#!g2.1
